In [112]:
pip install -q torch transformers langchain_chroma bitsandbytes langchain faiss-gpu langchain_huggingface langchain-community sentence-transformers  pacmap tqdm matplotlib datasets

In [113]:
pip install -q sentence_transformers

In [114]:
!python --version

Python 3.10.12


In [115]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import csv
import sys
import numpy as np
import time
import random
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
import textwrap
import torch

import random
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score

## Processing dataset

In [116]:
import json
from huggingface_hub import hf_hub_download

filepath = hf_hub_download(
    repo_id='McAuley-Lab/Amazon-C4',
    filename='sampled_item_metadata_1M.jsonl',
    repo_type='dataset'
)

item_pool = []
with open(filepath, 'r') as file:
    for line in file:
        item_pool.append(json.loads(line.strip()))

In [117]:
from datasets import load_dataset

dataset = load_dataset('McAuley-Lab/Amazon-C4')['test']

In [118]:
item_metadata_map = {item['item_id']: {'metadata': item['metadata'], 'category': item['category']} for item in item_pool}

In [119]:
# new_list = []
# for data in dataset:
#     item_id = data['item_id']
#     item_info = item_metadata_map.get(item_id, {'metadata': None, 'category': None})  # 找到 metadata 和 category
#     new_entry = {
#         'query': data['query'],
#         'item_id': item_id,
#         'metadata': item_info['metadata'],
#         'category': item_info['category']
#     }
#     new_list.append(new_entry)

In [120]:
new_list = []
for data in dataset:
    item_id = data['item_id']
    item_info = item_metadata_map.get(item_id, {'metadata': None, 'category': None})  # 找到 metadata 和 category

    # 检查 metadata 的长度
    metadata = item_info['metadata']
    if metadata is None or len(metadata.split()) < 10:  # 如果 metadata 为空或单词数少于 10
        continue  # 跳过该条数据

    # 构造新的条目
    new_entry = {
        'query': data['query'],
        'item_id': item_id,
        'metadata': metadata,
        'category': item_info['category']
    }
    new_list.append(new_entry)


In [121]:
print(len(new_list))

20250


In [122]:
# 初始化两个列表，分别存储 query 和 passage
queries = []
passages = []

# 计算前 1/10 的数据长度
limit = max(1, len(new_list) // 20)  # 确保至少取 1 条

# 遍历 new_list
for idx, entry in enumerate(new_list):
    # 仅处理前 1/10 的 query
    if idx < limit:
        queries.append(f"query: {entry['query']}")
        # 所有 passage 都添加
        passages.append(f"passage: {entry['metadata']}")

# 合并所有 query 和 passage 到一个列表中，确保 query 在前，passage 在后
input_texts = queries + passages

# 打印结果
# print(input_texts)


In [123]:
# 从 item_pool 中随机打乱并划分训练集与测试集
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

random.shuffle(new_list)  # 随机打乱
split_idx = int(0.9 * len(new_list))
train_pool = new_list[:split_idx]
test_pool = new_list[split_idx:]

# 提取 query 和 category，用于分类
def prepare_dataset(pool):
    queries = [item['metadata'] for item in pool if item['metadata'] and item['category']]
    categories = [item['category'] for item in pool if item['metadata'] and item['category']]
    return queries, categories

train_queries, train_categories = prepare_dataset(train_pool)
test_queries, test_categories = prepare_dataset(test_pool)

# 将 category 转换为索引
category_to_idx = {category: idx for idx, category in enumerate(set(train_categories))}
idx_to_category = {idx: category for category, idx in category_to_idx.items()}
train_labels = [category_to_idx[cat] for cat in train_categories]
test_labels = [category_to_idx[cat] for cat in test_categories]


In [124]:
class QueryDataset(Dataset):
    def __init__(self, queries, labels, tokenizer, max_len=128):
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        query = self.queries[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            query,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# 初始化 BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 创建 DataLoader
train_dataset = QueryDataset(train_queries, train_labels, tokenizer)
test_dataset = QueryDataset(test_queries, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [125]:
# 自动检测是否有可用的 GPU，如果没有，则使用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [126]:
class QueryClassifier(nn.Module):
    def __init__(self, num_categories):
        super(QueryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_categories)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # [CLS] token embedding
        logits = self.classifier(cls_token)
        return logits

# 初始化模型
num_categories = len(category_to_idx)
model = QueryClassifier(num_categories).to(device)

In [127]:
def train_model(model, train_loader, test_loader, num_epochs=3, lr=1e-5):
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    return model

model = train_model(model, train_loader, test_loader, num_epochs=3)


Training Epoch 1: 100%|██████████| 570/570 [02:09<00:00,  4.42it/s]


Epoch 1, Loss: 1.6271


Training Epoch 2: 100%|██████████| 570/570 [02:08<00:00,  4.42it/s]


Epoch 2, Loss: 0.6908


Training Epoch 3: 100%|██████████| 570/570 [02:08<00:00,  4.43it/s]

Epoch 3, Loss: 0.4623


In [128]:
def evaluate_model(model, test_loader, top_k=3):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask)
            _, top_preds = torch.topk(outputs, k=top_k, dim=-1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(top_preds.cpu().numpy())

    # 计算 Top-K 准确率
    top_k_accuracy = 0
    for label, preds in zip(all_labels, all_predictions):
        if label in preds:
            top_k_accuracy += 1

    top_k_accuracy /= len(all_labels)
    print(f"Top-{top_k} Accuracy: {top_k_accuracy:.4f}")

evaluate_model(model, test_loader, top_k=3)


Evaluating: 100%|██████████| 64/64 [00:08<00:00,  7.98it/s]

Top-3 Accuracy: 0.9580


In [129]:
from torch.nn.functional import softmax

# 新建一个空列表存储结果
result_list = []

# 将模型设置为评估模式
model.eval()

# 遍历测试集
with torch.no_grad():
    for batch in test_loader:
        # print(batch)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        # original_queries = batch["query"]  # 保存原始 query

        # 模型预测
        logits = model(input_ids, attention_mask)
        probabilities = softmax(logits, dim=-1)

        # 获取 top-3 类别
        top2_indices = torch.topk(probabilities, 2, dim=-1).indices.cpu().numpy()

        # 构造结果
        for top2 in top2_indices:
            # 获取 top-2 类别名称
            top2_categories = [idx_to_category[idx] for idx in top2]

            # 根据类别找到 `new_list` 中的 items
            matched_items = [
                {
                    "item_id": item["item_id"],
                    "metadata": item["metadata"],
                    "category": item["category"],
                }
                for item in new_list
                if item["category"] in top2_categories
            ]

            # 保存结果
            result_list.append({
                # "query": query,
                "top2_categories": top2_categories,
                "matched_items": matched_items
            })


In [130]:
for i, result in enumerate(result_list):
    result['query'] = test_pool[i]['query']
    result['real_category'] = test_pool[i]['category']
    result['real_item_id'] = test_pool[i]['item_id']

# 检查结果
print(result_list[0]['matched_items'])

[{'item_id': 'B0CBHH1FVK', 'metadata': 'Exercise Bike Stationary, CHAOKE Indoor Cycling Bike with Heavy Flywheel, Comfortable Seat Cushion, Silent Belt Drive and LCD Monitor for Home Gym Cardio Workout Training. .', 'category': 'Sports'}, {'item_id': 'B087PTNHMF', 'metadata': 'Air Intake Turbo Tube, Turbochagrer Intake Pipe Hose for Mercedes-Benz W172 SLK200 SLK250, W204 C180 C200 C250, W212 E200 E250 E250 M271 Engine, 2710901629 2710901729 2710902029 2711801819 2710901929. Air Intake Turbo Tube Intake Pipe Hose Replacement part number : 2710901629 2710901729 2710902029 2711801819 2710901929 Other part numbers : A2710901629 A2710901729 A2710902029 A2711801819 Fitment : for 2011-2016 Mercedes-Benz W172 SLK200 with M271 Engine for 2011-2016 Mercedes-Benz W172 SLK250 with M271 Engine for 2007-2013 Mercedes-Benz W204 C180 with M271 Engine for 2007-2013 Mercedes-Benz W204 C200 with M271 Engine for 2007-2013 Mercedes-Benz W204 C250 with M271 Engine for 2009-2016 Mercedes-Benz W212 E200 with 

In [131]:
# -*- coding: utf-8 -*-

import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import gc

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# 2. 初始化tokenizer和模型，并将模型移动到指定设备
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small').to(device)

In [132]:
for result in result_list:
    query = []
    passages = []
    query.append(f"query: {result['query']}")
    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")
    # print(query)
    input_texts = query + passages
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    embeddings = F.normalize(embeddings, p=2, dim=1)

    scores = (embeddings[:1] @ embeddings[1:].T) * 100
    print(scores)
    break


tensor([[80.7630, 78.2953, 76.5069,  ..., 77.1785, 80.3873, 80.8040]],
       device='cuda:0')


In [133]:
import torch
import torch.nn.functional as F

for result in result_list:
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Get top-10 scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=1000)

    # Map top scores to matched_items and check their IDs
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]
    print(result['real_item_id'])
    for i, matched_item in enumerate(top_matched_items):
        item_id = matched_item['item_id']
        is_real_item = item_id == result['real_item_id']
        print(f"Rank {i+1}: Score = {top_scores[i].item():.2f}, Item ID = {item_id}, Is Real Item: {is_real_item}")

    break  # Break after processing the first result (for debugging)


B0BP6WWSBD
Rank 1: Score = 85.68, Item ID = B07JLTMQJT, Is Real Item: False
Rank 2: Score = 85.63, Item ID = B0B6BD13Q9, Is Real Item: False
Rank 3: Score = 85.32, Item ID = B0B464RB6B, Is Real Item: False
Rank 4: Score = 85.14, Item ID = B0BFQ4YG6Z, Is Real Item: False
Rank 5: Score = 85.07, Item ID = B0923LNLK7, Is Real Item: False
Rank 6: Score = 84.90, Item ID = B09D2TRSHM, Is Real Item: False
Rank 7: Score = 84.71, Item ID = B0BP6WWSBD, Is Real Item: True
Rank 8: Score = 84.71, Item ID = B0BP6WWSBD, Is Real Item: True
Rank 9: Score = 84.69, Item ID = B0C5H87577, Is Real Item: False
Rank 10: Score = 84.65, Item ID = B09F35NTYT, Is Real Item: False
Rank 11: Score = 84.65, Item ID = B09F35NTYT, Is Real Item: False
Rank 12: Score = 84.61, Item ID = B08R2N5SDX, Is Real Item: False
Rank 13: Score = 84.59, Item ID = B097RWW2PX, Is Real Item: False
Rank 14: Score = 84.48, Item ID = B07D3PVBJ4, Is Real Item: False
Rank 15: Score = 83.48, Item ID = B0B4689MYZ, Is Real Item: False
Rank 16: S

In [134]:
print(len(result_list[2]['matched_items']))

3468


In [135]:
import torch
import torch.nn.functional as F
from tqdm import tqdm  # 引入 tqdm

real_item_in_top200_count = 0  # 记录 top-10 中包含 real_item 的次数
total_results = min(100, len(result_list))  # 只处理前 100 个结果

for result in tqdm(result_list[:100], desc="Processing Results", unit="result"):
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Get top-10 scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=200)

    # Map top scores to matched_items and check if real_item is in top-10
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]

    # Check if real_item is in top-10
    real_item_found = any(matched_item['item_id'] == result['real_item_id'] for matched_item in top_matched_items)

    # Update count if real_item is found in top-10
    if real_item_found:
        real_item_in_top200_count += 1

# Calculate the probability
if total_results > 0:
    probability = real_item_in_top200_count / total_results
else:
    probability = 0.0

print(f"\nProbability of real_item appearing in top-200: {probability:.2%}")


Processing Results: 100%|██████████| 100/100 [02:43<00:00,  1.63s/result]


Probability of real_item appearing in top-200: 79.00%


In [136]:
import torch
import torch.nn.functional as F
from tqdm import tqdm  # 引入 tqdm

real_item_in_top200_count = 0  # 记录 top-k 中包含 real_item 的次数
total_results = len(result_list)  # 总结果数

for result in tqdm(result_list, desc="Processing Results", unit="result"):  # 遍历所有结果
    query = []
    passages = []
    query.append(f"query: {result['query']}")

    for matched_item in result['matched_items']:
        passages.append(f"passage: {matched_item['metadata']}")

    # Combine query and passages into input_texts
    input_texts = query + passages

    # Tokenize inputs
    batch_dict = tokenizer(
        input_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    batch_dict = {key: value.to(device) for key, value in batch_dict.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute scores
    scores = (embeddings[:1] @ embeddings[1:].T) * 100  # Compute similarity scores
    scores = scores.squeeze(0)  # Remove extra dimension for easier processing

    # Dynamically determine top-k value
    k = min(200, scores.size(0))  # Ensure k does not exceed the number of scores

    # Get top-k scores and corresponding indices
    top_scores, top_indices = torch.topk(scores, k=k)

    # Map top scores to matched_items and check if real_item is in top-k
    top_matched_items = [result['matched_items'][idx] for idx in top_indices]

    # Check if real_item is in top-k
    real_item_found = any(matched_item['item_id'] == result['real_item_id'] for matched_item in top_matched_items)

    # Update count if real_item is found in top-k
    if real_item_found:
        real_item_in_top200_count += 1

# Calculate the probability
if total_results > 0:
    probability = real_item_in_top200_count / total_results
else:
    probability = 0.0

print(f"\nProbability of real_item appearing in top-200: {probability:.2%}")


Processing Results: 100%|██████████| 2025/2025 [53:34<00:00,  1.59s/result]


Probability of real_item appearing in top-200: 74.07%
